In this project,we will work with housing data for the city of Ames,Iowa,United States from 2006 to 2010.With the help of different features in the dataset,we will try to predict the sale price of a given house.

# Loading the Data

In [125]:
#importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold

In [31]:
housing=pd.read_csv('AmesHousing.tsv',delimiter='\t')
housing.head()
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
Order              2930 non-null int64
PID                2930 non-null int64
MS SubClass        2930 non-null int64
MS Zoning          2930 non-null object
Lot Frontage       2440 non-null float64
Lot Area           2930 non-null int64
Street             2930 non-null object
Alley              198 non-null object
Lot Shape          2930 non-null object
Land Contour       2930 non-null object
Utilities          2930 non-null object
Lot Config         2930 non-null object
Land Slope         2930 non-null object
Neighborhood       2930 non-null object
Condition 1        2930 non-null object
Condition 2        2930 non-null object
Bldg Type          2930 non-null object
House Style        2930 non-null object
Overall Qual       2930 non-null int64
Overall Cond       2930 non-null int64
Year Built         2930 non-null int64
Year Remod/Add     2930 non-null int64
Roof Style         29

In [32]:
import seaborn as sns
cor=housing.corr()
numerical_cols=housing.select_dtypes(include=['float','int'])
#numerical_train=housing.drop(columns=['PID','Year Built','Year Remod/Add','Garage Yr Blt','Mo Sold','Yr Sold'],axis=1)
null_val=numerical_cols.isnull().sum()
null_series=pd.Series(null_val)
null_series
#full_cols_series = null_series[null_series == 0]
#housing_subset=housing[full_cols_series.index]
#housing_subset
#cormat=housing_subset.corr()
#cormat
#sorted_corrs=cormat['SalePrice'].abs().sort_values()
#top_corrs=sorted_corrs[sorted_corrs>0.3]
#top_features=sorted_cor[sorted_cor>0.4]
#final_cor=housing_subset[top_corrs.index].corr()
#sns.heatmap(final_cor)



Lot Frontage      490
Mas Vnr Area       23
BsmtFin SF 1        1
BsmtFin SF 2        1
Bsmt Unf SF         1
Total Bsmt SF       1
Bsmt Full Bath      2
Bsmt Half Bath      2
Garage Yr Blt     159
Garage Cars         1
Garage Area         1
dtype: int64

In [33]:
def transform_features(df):
    return df
def selected_features(df):
    return df[['Gr Liv Area','SalePrice']]
def train_and_test(df):
    train=df[:1460]
    test=df[1460:]
    numeric_train=train.select_dtypes(include=['integer','float'])
    numeric_test=test.select_dtypes(include=['integer','float'])
    
    features = numeric_train.columns.drop("SalePrice")
    lr = LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = np.sqrt(mse)
    return rmse
data=transform_features(housing)
selection=selected_features(data)
error=train_and_test(selection)
error
    


57088.25161263909

The RMSE value is 57088 when the Gr Liv Area feature is selected which is the ground living area in square feet.We selected this feature because it showed some correlation with selling price values.Lets see if we can improve this by doing some feature engineering.

# Feature Engineering

In this section,we would be deciding which features are relevant for our model and which are not.Features with more number of missing values would be discarded.Any text columns with missing values would be discarded.Then,we will impute the remaining columns with most commmon (the mode) of each column respectively.After this,we would also try to modify the existing features so as it make them more useful for our model.Finally,we would remove the columns that can cause data leakage.

In [34]:
null=housing.isnull().sum()

In [35]:
drop_missing_cols = null[(null > len(housing)/20)].sort_values()
drop_missing_cols

Garage Type       157
Garage Yr Blt     159
Garage Finish     159
Garage Qual       159
Garage Cond       159
Lot Frontage      490
Fireplace Qu     1422
Fence            2358
Alley            2732
Misc Feature     2824
Pool QC          2917
dtype: int64

In [36]:
housing=housing.drop(drop_missing_cols.index,axis=1)

In [37]:
housing.isnull().sum()

Order               0
PID                 0
MS SubClass         0
MS Zoning           0
Lot Area            0
Street              0
Lot Shape           0
Land Contour        0
Utilities           0
Lot Config          0
Land Slope          0
Neighborhood        0
Condition 1         0
Condition 2         0
Bldg Type           0
House Style         0
Overall Qual        0
Overall Cond        0
Year Built          0
Year Remod/Add      0
Roof Style          0
Roof Matl           0
Exterior 1st        0
Exterior 2nd        0
Mas Vnr Type       23
Mas Vnr Area       23
Exter Qual          0
Exter Cond          0
Foundation          0
Bsmt Qual          80
                   ..
Electrical          1
1st Flr SF          0
2nd Flr SF          0
Low Qual Fin SF     0
Gr Liv Area         0
Bsmt Full Bath      2
Bsmt Half Bath      2
Full Bath           0
Half Bath           0
Bedroom AbvGr       0
Kitchen AbvGr       0
Kitchen Qual        0
TotRms AbvGrd       0
Functional          0
Fireplaces

In [38]:
text_mv_counts = housing.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]

housing= housing.drop(drop_missing_cols_2.index, axis=1)

In [39]:
num_missing = housing.select_dtypes(include=['int', 'float']).isnull().sum()
fixable_numeric_cols = num_missing[(num_missing < len(housing)/20) & (num_missing > 0)].sort_values()
fixable_numeric_cols.index
#df = df.apply(lambda x:x.fillna(x.value_counts().index[0]))

Index(['BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Garage Cars', 'Garage Area', 'Bsmt Full Bath', 'Bsmt Half Bath',
       'Mas Vnr Area'],
      dtype='object')

In [41]:
impute=SimpleImputer(strategy='most_frequent')
new_housing=pd.DataFrame(impute.fit_transform(housing))
new_housing.columns=housing.columns
new_housing.index=housing.index
new_housing

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900
5,6,527105030,60,RL,9978,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,6,2010,WD,Normal,195500
6,7,527127150,120,RL,4920,Pave,Reg,Lvl,AllPub,Inside,...,170,0,0,0,0,4,2010,WD,Normal,213500
7,8,527145080,120,RL,5005,Pave,IR1,HLS,AllPub,Inside,...,0,0,144,0,0,1,2010,WD,Normal,191500
8,9,527146030,120,RL,5389,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,236500
9,10,527162130,60,RL,7500,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,6,2010,WD,Normal,189000


In [42]:
new_housing.isnull().sum()

Order              0
PID                0
MS SubClass        0
MS Zoning          0
Lot Area           0
Street             0
Lot Shape          0
Land Contour       0
Utilities          0
Lot Config         0
Land Slope         0
Neighborhood       0
Condition 1        0
Condition 2        0
Bldg Type          0
House Style        0
Overall Qual       0
Overall Cond       0
Year Built         0
Year Remod/Add     0
Roof Style         0
Roof Matl          0
Exterior 1st       0
Exterior 2nd       0
Mas Vnr Area       0
Exter Qual         0
Exter Cond         0
Foundation         0
BsmtFin SF 1       0
BsmtFin SF 2       0
                  ..
Central Air        0
1st Flr SF         0
2nd Flr SF         0
Low Qual Fin SF    0
Gr Liv Area        0
Bsmt Full Bath     0
Bsmt Half Bath     0
Full Bath          0
Half Bath          0
Bedroom AbvGr      0
Kitchen AbvGr      0
Kitchen Qual       0
TotRms AbvGrd      0
Functional         0
Fireplaces         0
Garage Cars        0
Garage Area  

In [44]:
years_sold = new_housing['Yr Sold'] - new_housing['Year Built']
years_sold[years_sold < 0]

2180    -1
dtype: object

In [45]:
years_remod=new_housing['Yr Sold'] - new_housing['Year Remod/Add']
years_remod[years_remod<0]

1702    -1
2180    -2
2181    -1
dtype: object

In [47]:
new_housing['Years Before Sale'] = years_sold
new_housing['Years Since Remod'] = years_remod

new_housing = new_housing.drop([1702, 2180, 2181], axis=0)

## No longer need original year columns
new_housing = new_housing.drop(["Year Built", "Year Remod/Add"], axis = 1)
new_housing.columns

Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Area', 'Street',
       'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config', 'Land Slope',
       'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Area',
       'Exter Qual', 'Exter Cond', 'Foundation', 'BsmtFin SF 1',
       'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', 'Heating', 'Heating QC',
       'Central Air', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Garage Cars',
       'Garage Area', 'Paved Drive', 'Wood Deck SF', 'Open Porch SF',
       'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val',
       'Mo Sold', 'Yr Sold', 'Sale Type', 'Sale Condition', 'SalePrice',
   

In [65]:
#new_housing = new_housing.drop(["PID", "Order"], axis=1)

## Drop columns that leak info about the final sale
#new_housing = new_housing.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
new_housing['Years Before Sale']

0       50
1       49
2       52
3       42
4       13
5       12
6        9
7       18
8       15
9       11
10      17
11      18
12      12
13      20
14      25
15       7
16      22
17       0
18      59
19      32
20      33
21      36
22      10
23      40
24      39
25      42
26      40
27      39
28      11
29      39
        ..
2900     1
2901     1
2902     1
2903    55
2904     9
2905     8
2906     8
2907     0
2908    29
2909    29
2910    33
2911    38
2912    36
2913    36
2914    34
2915    37
2916    36
2917    36
2918    36
2919    36
2920    36
2921    30
2922    30
2923    29
2924    46
2925    22
2926    23
2927    14
2928    32
2929    13
Name: Years Before Sale, Length: 2927, dtype: object

In [67]:
replacement_values_dict = housing[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]

In [68]:
housing = housing.fillna(replacement_values_dict)

0    64
dtype: int64

In [69]:
years_s = housing['Yr Sold'] - housing['Year Built']
years_s[years_s < 0]

2180   -1
dtype: int64

In [70]:
years_since_r = housing['Yr Sold'] - housing['Year Remod/Add']
housing['Years Before Sale'] = years_s
housing['Years Since Remod'] = years_since_r

## Drop rows with negative values for both of these new features
housing = housing.drop([1702, 2180, 2181], axis=0)

## No longer need original year columns
housing = housing.drop(["Year Built", "Year Remod/Add"], axis = 1)

0       50
1       49
2       52
3       42
4       13
5       12
6        9
7       18
8       15
9       11
10      17
11      18
12      12
13      20
14      25
15       7
16      22
17       0
18      59
19      32
20      33
21      36
22      10
23      40
24      39
25      42
26      40
27      39
28      11
29      39
        ..
2900     1
2901     1
2902     1
2903    55
2904     9
2905     8
2906     8
2907     0
2908    29
2909    29
2910    33
2911    38
2912    36
2913    36
2914    34
2915    37
2916    36
2917    36
2918    36
2919    36
2920    36
2921    30
2922    30
2923    29
2924    46
2925    22
2926    23
2927    14
2928    32
2929    13
Name: Years Before Sale, Length: 2927, dtype: int64

In [71]:
#housing = housing.drop(["PID", "Order"], axis=1)

## Drop columns that leak info about the final sale
#housing = housing.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2927 entries, 0 to 2929
Data columns (total 58 columns):
MS SubClass          2927 non-null int64
MS Zoning            2927 non-null object
Lot Area             2927 non-null int64
Street               2927 non-null object
Lot Shape            2927 non-null object
Land Contour         2927 non-null object
Utilities            2927 non-null object
Lot Config           2927 non-null object
Land Slope           2927 non-null object
Neighborhood         2927 non-null object
Condition 1          2927 non-null object
Condition 2          2927 non-null object
Bldg Type            2927 non-null object
House Style          2927 non-null object
Overall Qual         2927 non-null int64
Overall Cond         2927 non-null int64
Roof Style           2927 non-null object
Roof Matl            2927 non-null object
Exterior 1st         2927 non-null object
Exterior 2nd         2927 non-null object
Mas Vnr Area         2927 non-null float64
Exter Qual    

Now,we are done experimenting and now we will be modifying the transform_features function.

In [84]:
selected=housing.select_dtypes(include=['int64','float64'])
for i in selected.columns:
    new_housing[i]=new_housing[i].astype(float)
new_housing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2927 entries, 0 to 2929
Data columns (total 58 columns):
MS SubClass          2927 non-null float64
MS Zoning            2927 non-null object
Lot Area             2927 non-null float64
Street               2927 non-null object
Lot Shape            2927 non-null object
Land Contour         2927 non-null object
Utilities            2927 non-null object
Lot Config           2927 non-null object
Land Slope           2927 non-null object
Neighborhood         2927 non-null object
Condition 1          2927 non-null object
Condition 2          2927 non-null object
Bldg Type            2927 non-null object
House Style          2927 non-null object
Overall Qual         2927 non-null float64
Overall Cond         2927 non-null float64
Roof Style           2927 non-null object
Roof Matl            2927 non-null object
Exterior 1st         2927 non-null object
Exterior 2nd         2927 non-null object
Mas Vnr Area         2927 non-null float64
Exter 

In [72]:
def transform_features1(df):
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df)/20)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis=1)
    return df


In [90]:
def transform_features(df):
    from sklearn.impute import SimpleImputer
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df)/20)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    impute=SimpleImputer(strategy='most_frequent')
    new_df=pd.DataFrame(impute.fit_transform(df))
    new_df.columns=df.columns
    new_df.index=df.index
    
    selected=df.select_dtypes(include=['int64','float64'])
    for i in selected.columns:
        new_df[i]=new_df[i].astype(float)
    
    years_sold = new_df['Yr Sold'] - new_df['Year Built']
    years_since_remod = new_df['Yr Sold'] - new_df['Year Remod/Add']
    new_df['Years Before Sale'] = years_sold
    new_df['Years Since Remod'] = years_since_remod
    new_df = new_df.drop([1702, 2180, 2181], axis=0)

    new_df = new_df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis=1)
    return new_df
    

In [97]:
def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

def train_and_test(df):  
    train = df[:1460]
    test = df[1460:]
    
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    features = numeric_train.columns.drop("SalePrice")
    lr = LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = np.sqrt(mse)
    
    return rmse
def train_test(df):
    train=df[:1460]
    test=df[1460:]
    numeric_train = train.select_dtypes(include=['float64'])
    numeric_test = test.select_dtypes(include=['float64'])
    
    features = numeric_train.columns.drop("SalePrice")
    lr = LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = np.sqrt(mse)
    return rmse
    
    
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
#filtered_df.columns
rmse = train_test(filtered_df)

rmse

55275.36731241307

In [100]:
float_df=transform_df.select_dtypes(include=['float64'])
float_df.head()

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Yr Sold,SalePrice,Years Before Sale,Years Since Remod
0,20.0,31770.0,6.0,5.0,112.0,639.0,0.0,441.0,1080.0,1656.0,...,62.0,0.0,0.0,0.0,0.0,0.0,2010.0,215000.0,50.0,50.0
1,20.0,11622.0,5.0,6.0,0.0,468.0,144.0,270.0,882.0,896.0,...,0.0,0.0,0.0,120.0,0.0,0.0,2010.0,105000.0,49.0,49.0
2,20.0,14267.0,6.0,6.0,108.0,923.0,0.0,406.0,1329.0,1329.0,...,36.0,0.0,0.0,0.0,0.0,12500.0,2010.0,172000.0,52.0,52.0
3,20.0,11160.0,7.0,5.0,0.0,1065.0,0.0,1045.0,2110.0,2110.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2010.0,244000.0,42.0,42.0
4,60.0,13830.0,5.0,5.0,0.0,791.0,0.0,137.0,928.0,928.0,...,34.0,0.0,0.0,0.0,0.0,0.0,2010.0,189900.0,13.0,12.0


In [103]:
c=float_df.corr()
cor_target=c['SalePrice'].abs().sort_values()
filter_cor=cor_target[cor_target>0.4]

BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64

In [104]:
transform_df = transform_df.drop(cor_target[cor_target < 0.4].index, axis=1)

In [106]:
cat_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

In [116]:
transform_cat_cols = []
for col in cat_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)
value_count={}
for cat in transform_cat_cols:
    value_count[cat]=transform_df[cat].value_counts()
drop_col_list=[]
for key,val in value_count.items():
    if len(val)>10:
        drop_col_list.append(key)
transform_df = transform_df.drop(drop_col_list, axis=1)  

Index(['MS Zoning', 'Street', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Roof Style', 'Roof Matl',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation',
       'BsmtFin SF 1', 'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air',
       '1st Flr SF', 'Gr Liv Area', 'Full Bath', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Garage Cars',
       'Garage Area', 'Paved Drive', 'SalePrice', 'Years Before Sale',
       'Years Since Remod'],
      dtype='object')

In [118]:
def select_features1(df,coeff_threshold=0.4,unique_threshold=10):
    numerical_df = df.select_dtypes(include=['float64'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    value_count={}
    for cat in transform_cat_cols:
        value_count[cat]=df[cat].value_counts()
    drop_col_list=[]
    for key,val in value_count.items():
        if len(val)>unique_threshold:
            drop_col_list.append(key)
    df = df.drop(drop_col_list, axis=1)  
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1)
    
    return df


In [133]:
def train_test_k(df, k=0):
    numeric_df = df.select_dtypes(include=['float64'])
    features = numeric_df.columns.drop("SalePrice")
    lr = LinearRegression()
    
    if k == 0:
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    if k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        min_rmse=np.min(rmse_values)
        return min_rmse

In [134]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features1(transform_df)
#filtered_df.columns
rmse = train_test_k(filtered_df,k=4)

rmse

[34303.31773646276, 38966.523342673994, 30639.72633722792, 29929.189787510855]


29929.189787510855